In [19]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [20]:
import os
import http.client
import json

In [21]:
# ! pip install -U langchain-groq

In [22]:
# pick a model
from langchain.chat_models import init_chat_model
llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

In [23]:
from langchain_core.tools import tool

@tool
def browse(txt: str) -> str:
    """ Call this tool if you want to search the web for current information"""

    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
    "q": txt,
    "gl": "in"
    })
    headers = {
    'X-API-KEY': os.environ.get("SERPER_API_KEY"),
    'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read()
    #print(data.decode("utf-8"))
    data_dict = json.loads(data.decode("utf-8"))
    return data_dict['organic'][0]['snippet']


In [24]:
tools_list = [browse]
tools_dict = {t.name: t for t in tools_list} # comes in handy at the time of invokation

In [25]:
# we create a tool calling Agent by binding a list of tools to the llm
llm_with_tools = llm.bind_tools(tools_list)

In [26]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# This will store all converation
chat_history = [
    SystemMessage(content="You are a chatbot. Depending on the conversation, if user demands current information, you must use the tool 'browse' to get the information. If you do not need current information, you can answer directly. Always try to keep the conversation short and crisp.")
    ]

In [27]:
chat_history=[]

In [28]:
chat_history.append(HumanMessage(content="Tell me about the French Revolution"))

response = llm_with_tools.invoke(chat_history)
response


AIMessage(content="The French Revolution was a pivotal event in modern history that took place from 1789 to 1799. It was a complex and multifaceted phenomenon that involved the overthrow of the monarchy, the establishment of a new political and social order, and the rise of Napoleon Bonaparte. The Revolution was sparked by a combination of factors, including financial crisis, food shortages, and the influence of the Enlightenment ideas of liberty, equality, and fraternity.\n\nThe Revolution began with the Storming of the Bastille on July 14, 1789, which marked the beginning of the end of the absolute monarchy and the rise of the Third Estate, which represented the common people. The National Assembly, which was formed in 1789, adopted the Declaration of the Rights of Man and of the Citizen, which enshrined the principles of liberty, equality, and fraternity.\n\nThe early years of the Revolution saw the establishment of a constitutional monarchy, with King Louis XVI as the head of state

In [29]:
chat_history.append(response)
chat_history


[HumanMessage(content='Tell me about the French Revolution', additional_kwargs={}, response_metadata={}),
 AIMessage(content="The French Revolution was a pivotal event in modern history that took place from 1789 to 1799. It was a complex and multifaceted phenomenon that involved the overthrow of the monarchy, the establishment of a new political and social order, and the rise of Napoleon Bonaparte. The Revolution was sparked by a combination of factors, including financial crisis, food shortages, and the influence of the Enlightenment ideas of liberty, equality, and fraternity.\n\nThe Revolution began with the Storming of the Bastille on July 14, 1789, which marked the beginning of the end of the absolute monarchy and the rise of the Third Estate, which represented the common people. The National Assembly, which was formed in 1789, adopted the Declaration of the Rights of Man and of the Citizen, which enshrined the principles of liberty, equality, and fraternity.\n\nThe early years of 

In [31]:
# Since there was a tool call, execute the tool and append the tool output in the converation

if response.tool_calls:
    chat_history.append(tools_dict[response.tool_calls[0]["name"]].invoke(response.tool_calls[0]))
chat_history

[HumanMessage(content='Tell me about the French Revolution', additional_kwargs={}, response_metadata={}),
 AIMessage(content="The French Revolution was a pivotal event in modern history that took place from 1789 to 1799. It was a complex and multifaceted phenomenon that involved the overthrow of the monarchy, the establishment of a new political and social order, and the rise of Napoleon Bonaparte. The Revolution was sparked by a combination of factors, including financial crisis, food shortages, and the influence of the Enlightenment ideas of liberty, equality, and fraternity.\n\nThe Revolution began with the Storming of the Bastille on July 14, 1789, which marked the beginning of the end of the absolute monarchy and the rise of the Third Estate, which represented the common people. The National Assembly, which was formed in 1789, adopted the Declaration of the Rights of Man and of the Citizen, which enshrined the principles of liberty, equality, and fraternity.\n\nThe early years of 

In [32]:
chat_history.append(HumanMessage(content="What is 10 plus 10?"))

response = llm_with_tools.invoke(chat_history)
response

AIMessage(content='20', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 749, 'total_tokens': 751, 'completion_time': 0.014609267, 'prompt_time': 0.062616895, 'queue_time': 0.218008993, 'total_time': 0.077226162}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b8fe596f-6327-408b-87f9-b9e34bebe11c-0', usage_metadata={'input_tokens': 749, 'output_tokens': 2, 'total_tokens': 751})